In [2]:
# loading modules for creating models (Tensorflow/Keras Framework)
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, BatchNormalization, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import datetime
import pathlib

print("modules loaded successfully")
print(f"Tensorflow Version : {tf.__version__}")

modules loaded successfully
Tensorflow Version : 2.17.0


In [10]:
# Raw Dataset Directory
train_data_dir = pathlib.Path("../data/train_set")
test_data_dir = pathlib.Path("../data/test_set")

# Getting a count of images in both train and test set
train_image_count = len(list(train_data_dir.glob("*/*.jpg")))
test_image_count = len(list(test_data_dir.glob("*/*.jpg")))

print(f"train size : {train_image_count}")
print(f"test size : {test_image_count}\n")


# class names in dataset
CLASS_NAMES = np.array([item.name for item in train_data_dir.glob('*')])
print(CLASS_NAMES,"\n")

# obtain number of classes
num_classes = len(CLASS_NAMES)
print(f"Number of classes : {num_classes}")

train size : 819
test size : 212

['BA- cellulitis' 'BA-impetigo' 'FU-athlete-foot' 'FU-nail-fungus'
 'FU-ringworm' 'PA-cutaneous-larva-migrans' 'VI-chickenpox' 'VI-shingles'] 

Number of classes : 8


In [ ]:
# Generating data from ImageGenerator function in preprocessing class of tensorflow, data pulled from Kaggle for auxillary training

# Shape inputs of neural net model
BATCH_SIZE =32 # Used for better utilization of memory
IMG_HEIGHT = 227 # Define Image Height
IMG_WIDTH = 227 # Define Image Width

# Instantiate ImageGenerator class with rescale parameter
image_generator = ImageDataGenerator(rescale=1./255) # RGB channels rescaled so pixel intensities are scaled between 0 and 1

#train data for model

def data_generator(HEIGHT : int =227, WIDTH : int =227, BATCH_SIZE : int=32):
    """
    Generates image datasets for input into a Neural Network model. 
    
    Args:
        height (int) : Height of image in pixels 
                        Default is 227
        width (int) : Width of image in pixels 
                        Default is 227
        batch_size (int): A list of metrics to be evaluated by the model during training and testing. 
                        Default is 32.

    Returns:
            Tensorflow image dataset
    """

data_gen = image_generator.flow_from_director(directory=str(train_data_dir),
                                                    batch_size = BATCH_SIZE,
                                                    shuffle = True,
                                                    target_size = (IMG_HEIGHT))


In [ ]:
# Creating an alex net model from scratch to test performance on a blanket model
def create_alexnet(num_classes : int, loss_function="categorical_crossentropy", metrics : list = ['accuracy']) -> tf.keras.Model :
    
    """
    Creates a Convolutional Neural Network (CNN) model based on the AlexNet architecture, 
    with Batch Normalization layers for stabilizing the training. The model starts with an input
    image dimension of (227, 227, 3) and progressively reduces the spatial dimensions while 
    increasing the number of filters in each Conv2D/MaxPooling block.

    Args:
        num_classes (int): The number of output classes for the classification task.
        loss_function (str or callable): The loss function to use during model compilation. 
                                        Default is 'categorical_crossentropy'.
        metrics (list): A list of metrics to be evaluated by the model during training and testing. 
                        Default is ['accuracy'].
        
    Returns:
        model (tf.keras.Model): A compiled CNN model with batch normalization, ready for training.
    
    Example:
        model = create_alexnet(10, loss_function='categorical_crossentropy')
        model.summary()
    """
    model = Sequential([

        # 1st convolutional-pooling block
        Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation="relu", input_shape=(227,227,3)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),

        # 2nd convolutional-pooling block
        Conv2D(filters=256, kernel_size=(5,5), strides=(2,2), activation="relu", padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),

        # 3rd convolutional block
        Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation="relu", padding='same'),
        BatchNormalization(),
        Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation="relu", padding='same'),
        BatchNormalization(),
        Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation="relu", padding='same'),
        BatchNormalization(),

        # Flatten layer to convert 3D feature map into 1D Vector to feed into Neural Network
        Flatten(),

        # Fully connected (dense) layers block with dropout regularization
        Dense(4096, activation="relu"),
        Dropout(0.5),
        Dense(4096, activation="relu"),
        Dropout(0.5),
        Dense(num_classes, activation="softmax")
    ])

    model.compile(optimizer='Adam', loss=loss_function, metrics=['accuracy'])

    return model

In [ ]:
model_A = create_alexnet()